In [1]:
import sys
import os
import time

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import ee
import geemap

ee.Initialize(project="thurgau-irrigation")

In [3]:
from wapor_et_processing import load_wapor_et_data
from utils.date_utils import print_collection_dates, create_centered_date_ranges
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset, aggregate_to_monthly, print_value_ranges

from utils.export_image_collection import export_collection_to_assets


from typing import List

---

In [4]:
# Little square around Oensingen:
oensingen_coordinates = [
    [
        [7.569238717890812, 47.35358169812031],
        [7.569238717890812, 47.21413609771895],
        [7.879806798836398, 47.21413609771895],
        [7.879806798836398, 47.35358169812031],
        [7.569238717890812, 47.35358169812031],
    ]
]

# Create an ee.Geometry object from the coordinates
oensingen_polygon = ee.Geometry.Polygon(oensingen_coordinates)
aoi_simple = oensingen_polygon.simplify(500)
aoi_buffered = aoi_simple.buffer(1000)

In [5]:
# Load WAPOR ET data
first_year = 2018
last_year = 2022
wapor_et_data = load_wapor_et_data(
    first_year, last_year, frequency="dekadal"
).filterBounds(aoi_buffered)

In [6]:
# print_collection_dates(wapor_et_data)

In [7]:
# SCALE = wapor_et_data.first().projection().nominalScale().getInfo()

# print(f"The scale of the data is {SCALE}")

# for year in range(2018, 2023):
#     et_wapor_year = wapor_et_data.filterDate(f"{year}-01-01", f"{year}-12-31")

#     et_wapor_year_list = et_wapor_year.toList(et_wapor_year.size())

#     for i in range(36):
#         dekadal = i % 3 + 1
#         month = i // 3 + 1

#         image_to_export = ee.Image(et_wapor_year_list.get(i))

#         task_name = f"Thurgau_ET_WAPOR_300m_{year}_{month}_{dekadal}"
#         asset_id = f"projects/thurgau-irrigation/assets/ET_WaPOR_300m_Oensingen_2018-2022/{task_name}"

#         export_image_to_asset(
#             image=image_to_export,
#             asset_id=asset_id,
#             task_name=task_name,
#             year=year,
#             scale=SCALE,
#             aoi=aoi_buffered,
#         )

#         print(f"Exported image {task_name} to asset {asset_id}")

#     print(f"Done exporting year {year}, sleeping for 15 seconds")
#     time.sleep(15)

# print("Done exporting all images, mate!")

## Sanity check

In [8]:
Map = geemap.Map()

vis_params = {
    "bands": ['ET'],
    "min": 0,
    "max": 5,
    "palette": ["blue", "green", "red"],
}

image = ee.Image(wapor_et_data.toList(wapor_et_data.size()).get(20))

Map.addLayer(image, vis_params, "ET")
Map.centerObject(aoi_buffered, 10)
Map


Map(center=[47.2839284121097, 7.724522758357868], controls=(WidgetControl(options=['position', 'transparent_bg…